In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate as sint

In [2]:
airfoil = np.loadtxt("NRELs826.txt", delimiter=",")
centroid = np.mean(airfoil, axis=0)

In [ ]:
class TrailingEdgeFix:
    """
        Fixes leading edge problem on airfoils for meshing, 
        and CFD applications. Main idea taken from PyAero
        airfoil analysis.
    
    """
    
    def __init__(self, airfoilpath): 
        
        self.airfoilpath = airfoilpath
        self.get_AirfoilSpline(150)
        self.calc_CurvatureProps()
        self.fix_TrailingEdge()
        
    def get_AirfoilSpline(self, points=300)->None: 
        """
            Parametrizes an airfoil based on a set points
            using Cubic Spline Interpolation.

        """
        try:
            self.airfoildata = np.loadtxt(self.airfoilpath, delimiter=",")
        except Exception as e:
            print(e)
        
        self.spl = sint.splprep(self.airfoildata.T, s=0.0, k=2)
        self.t = np.linspace(0, 1, points)
        self.coords = np.array(sint.splev(self.t, self.spl[0], der=0))
        self.d1 = sint.splev(self.t, self.spl[0], der=1)
        self.d2 = sint.splev(self.t, self.spl[0], der=2)
    
    def calc_CurvatureProps(self):
        """
            Calculates airfoil curve properties. 
        """
        x = self.coords[0]
        y = self.coords[1]
        x1d = self.d1[0]
        y1d = self.d1[1]
        x2d = self.d2[0]
        y2d = self.d2[1]
        
        num = x1d*y2d-y1d*x2d
        denom = (x1d**2+y1d**2)
        
        self.kappa = num/(denom**(3/2)) # curvature
        self.R = 1/self.kappa #radius of curvature
        
        self.xc = x - self.R*x1d/np.sqrt(denom)
        self.yc = y + self.R*y1d/np.sqrt(denom)
        
        self.leid = np.where(self.R == np.min(self.R))
        self.xle = self.xc[self.leid[0][0]]
        self.yle = self.yc[self.leid[0][0]]
               

    def refine_leading_edge(self):
        
        up_airfoil = self.coords[:, :self.leid[0][0]+1].T
        down_airfoil = self.coords[:,self.leid[0][0]:].T
    
    

        
nrel = TrailingEdgeFix("NRELs826.txt")


/tmp/ipykernel_830660/3372395252.py:98: RuntimeWarning: invalid value encountered in power
  x[i] = x[i]+signum*n[0]*shift_blend**ex*shift
/tmp/ipykernel_830660/3372395252.py:99: RuntimeWarning: invalid value encountered in power
  y[i] = y[i]+signum*n[1]*shift_blend**ex*shift
